In [ ]:
!pip install datasets transformers

# IMDB 데이터 세트 불러오기

In [ ]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")

# imdb 데이터 세트 토큰화

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# IMDB 데이터 세트 일부(샘플 100개) 적재
imdb_dataset = load_dataset("imdb", split="train[:100]")

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 자르기와 패딩을 사용해 IMDB 데이터 세트 토큰화
tokenized_imdb_dataset = imdb_dataset.map(
lambda x: tokenizer(x["text"], truncation=True, padding="max_length")
)

print(tokenized_imdb_dataset)

# 토큰의 첫 번째 행 가져오기
first_row_tokens = tokenized_imdb_dataset[0]["input_ids"]

# 처음 10개 토큰과 토큰에 해당하는 단어 출력
for token in first_row_tokens[:10]:
  print(f"토큰: {token}, 단어: {tokenizer.decode([token])}")

# Spaces 예제 코드
Gradio 인터페이스 설정

In [ ]:
!pip install gradio transformers

In [ ]:
import gradio as gr
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def sentiment_analysis(text):
    result = sentiment_pipeline(text)
    return result[0]["label"]

iface = gr.Interface(fn=sentiment_analysis, inputs="text", outputs="text")
iface.launch()

# 일래스틱 Eland

In [ ]:
!pip install eland elasticsearch seaborn

# 인덱스 생성 예제

## 일래스틱서치에 접속하여 샘플 인덱스 생성

In [ ]:
import getpass
from elasticsearch import Elasticsearch
from datetime import datetime

es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_key = getpass.getpass('Enter cluster API key:  ')

es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=es_api_key
                   )
es.info() # 클러스터 정보 확인

In [ ]:
mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "some_field": {"type": "float"},
            "column_a": {"type": "float"},
            "column_b": {"type": "float"},
            "category": {"type": "keyword"},
            "value": {"type": "float"}
        }
    }
}

# 인덱스 생성
es.indices.create(index="sample_eland_index", body=mapping)

# 인덱스에 샘플 데이터 넣기
documents = [
    {"some_field": 95.0, "column_a": 5.0, "column_b": 10.0, "category": "A", "value": 50.0},
    {"some_field": 150.0, "column_a": 7.0, "column_b": 20.0, "category": "B", "value": 140.0},
    {"some_field": 200.0, "column_a": 8.0, "column_b": 25.0, "category": "A", "value": 200.0},
    {"some_field": 50.0, "column_a": 4.0, "column_b": 12.5, "category": "C", "value": 50.0}
]

for doc in documents:
    es.index(index="sample_eland_index", body=doc)

## Eland 예제

In [ ]:
import eland as ed

df = ed.DataFrame(es_client=es, es_index_pattern="sample_eland_index")
filtered_df = df[df['some_field'] > 100]
filtered_df

In [ ]:
average_value = df['some_field'].mean()
average_value

In [ ]:
import seaborn as sns
import pandas as pd

filtered_df = df[df['some_field'] > 100]
pandas_df = filtered_df.to_pandas()
sns.boxplot(x='category', y='value', data=pandas_df)

# 허깅 페이스의 모델을 일래스틱서치에서 불러오기

In [ ]:
pip -q install eland elasticsearch transformers sentence_transformers torch==1.13

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient
import getpass

In [ ]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_key = getpass.getpass('Enter cluster API key:  ')

In [ ]:
# 일래스틱 클라우드 연결
es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=es_api_key
                   )
es.info() # 클러스터 정보 확인

In [ ]:
hf_model_id='sentence-transformers/msmarco-MiniLM-L-12-v3'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

In [ ]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

In [ ]:
# 일래스틱서치에 존재하는 모델 정보 조회
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

In [ ]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

In [ ]:
docs =  [
    {
      "text_field": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
    }
  ]

In [ ]:
z = MlClient.infer_trained_model(es, model_id=es_model_id, docs=docs)

In [ ]:
doc_0_vector = z['inference_results'][0]['predicted_value']
doc_0_vector

# 차원 축소

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA

In [ ]:
# Iris 데이터 세트 적재
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 차원 축소를 위해 PCA 적용
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
# 원 데이터 시각화
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.title('Original Iris dataset')
plt.show()

In [ ]:
# 차원 축소된 데이터 시각화
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Iris dataset after PCA')
plt.show()


# 양자화

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [ ]:
# 숫자 데이터 세트 불러오기
digits = datasets.load_digits()
X = digits.data

# 원본 데이터 세트에서 첫 번째 예시 출력
print("Original dataset (first example):\n", X[0])

In [ ]:
# 차원 축소를 위해 PCA 적용
pca = PCA(n_components=10)
X_reduced = pca.fit_transform(X)

# PCA 적용 후 첫 번째 예시 출력
print("\nReduced dataset after PCA (first example):\n", X_reduced[0])

In [ ]:
# 축소된 벡터를 [0, 255] 범위로 정규화
scaler = MinMaxScaler((0, 255))
X_scaled = scaler.fit_transform(X_reduced)

# 정규화 후 첫 번째 예시 출력
print("\nScaled dataset after normalization (first example):\n", X_scaled[0])

In [ ]:
# 스케일링 된 벡터를 8비트 정수로 양자화
X_quantized = np.round(X_scaled).astype(np.uint8)

# 양자화 후 첫 번째 예시 출력
print("\nQuantized dataset (first example):\n", X_quantized[0])